In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

from estudo import Estudo

dic, erros = Estudo(
    ano = 2024,
    id_modulo = [58],
    analito = None
).run()

In [ ]:
colunas = ['A24M10I1',	'A24M10I2',	'id_modulo','analito','part','regiao','sistema','eps_calculado','identico','cluster', 'A24M10I3',	'A24M1I1',	'A24M1I2',	'A24M1I3',	'cluster',	'A24M4I2',	'A24M4I3',	'A24M7I1',	'A24M7I2',	'safasf',]
import re

encontrar_col_item = lambda x: x if re.compile(r'A[0-9]+M[0-9]+I[0-9]+').findall(x) != [] else None

colunas_item = list(filter(encontrar_col_item, colunas))

colunas_item


['A24M10I1',
 'A24M10I2',
 'A24M10I3',
 'A24M1I1',
 'A24M1I2',
 'A24M1I3',
 'A24M4I2',
 'A24M4I3',
 'A24M7I1',
 'A24M7I2']

In [11]:
colunas_item = ['A24M10I1',	'A24M10I2',	'A24M10I3',	'A24M1I1',	'A24M1I2',	'A24M1I3',	'A24M4I1',	'A24M4I2',	'A24M4I3',	'A24M7I1',	'A24M7I2',	'A24M7I3',]
lista[14][['id_modulo','analito','part','regiao','sistema','eps_calculado','identico','cluster']+colunas_item]#.groupby('cluster').identico.agg(lambda x: set(x))

,id_modulo,analito,part,regiao,sistema,eps_calculado,identico,cluster,A24M10I1,A24M10I2,A24M10I3,A24M1I1,A24M1I2,A24M1I3,A24M4I1,A24M4I2,A24M4I3,A24M7I1,A24M7I2,A24M7I3
0,58,Glicose,235,RJ,1,0.024375,Sim,0,144.0,101.0,209.0,291.0,288.0,95.0,101.0,308.0,192.0,280.0,94.0,94.0
1,58,Glicose,5756,RJ,1,0.024375,Sim,0,144.0,101.0,209.0,291.0,288.0,95.0,101.0,308.0,192.0,280.0,94.0,94.0
2,58,Glicose,553,RJ,1,0.024375,Sim,1,144.0,98.0,192.0,264.0,271.0,85.0,102.0,299.0,200.0,294.0,90.0,98.0
3,58,Glicose,904,RJ,1,0.024375,Sim,1,144.0,98.0,192.0,264.0,271.0,85.0,102.0,299.0,200.0,294.0,90.0,98.0
4,58,Glicose,591,ES,1,0.024375,Sim,2,139.0,98.0,195.0,280.0,290.0,94.0,99.0,290.0,197.0,282.0,93.0,98.0
5,58,Glicose,806,ES,1,0.024375,Sim,2,139.0,98.0,195.0,280.0,290.0,94.0,99.0,290.0,197.0,282.0,93.0,98.0
6,58,Glicose,818,SP,1,0.024375,Sim,3,134.0,93.0,188.0,271.0,287.0,92.0,97.0,293.0,200.0,304.0,86.0,95.0
7,58,Glicose,1881,SP,1,0.024375,Sim,3,134.0,93.0,188.0,271.0,287.0,92.0,97.0,293.0,200.0,304.0,86.0,95.0
8,58,Glicose,1961,RS,1,0.024375,Não,4,140.0,98.0,197.0,274.0,285.0,87.0,105.0,298.0,196.0,294.0,96.0,101.0
9,58,Glicose,7343,RS,1,0.024375,Não,4,140.0,98.0,199.0,274.0,285.0,87.0,105.0,298.0,196.0,294.0,96.0,101.0


In [12]:
# Quantidade de módulos e analitos
from utils import Data
Data().get_data(ano = 2024, id_modulo = None, analito = None)[['id_modulo','analito']].drop_duplicates()

,id_modulo,analito
0,220,Valor (U/mL)
1070,577,Valor (UI/mL=KUI/L)
2360,724,CO2 Total
2362,724,Bicarbonato
2364,724,D-Xilose
...,...,...
1282237,24,Cálcio Total
1282242,24,Proteínas Totais
1282244,24,Glicose
1282992,24,Creatinina


In [2]:
from utils import Data

data = Data()
# parceiros = data.get_parceiros()
# modulos = data.get_modulos()
df = data.get_data(ano = 2024, id_modulo = [362], analito = ['Fosfatase Ácida Total'])

from utils import PreProcessamento

df_tratado = PreProcessamento(df).get_data()
df_tratado

item,part,id_modulo,analito,sistema,A24M10I1,A24M10I2,A24M10I3,A24M1I1,A24M1I2,A24M1I3,A24M4I1,A24M4I2,A24M4I3,A24M7I1,A24M7I2,A24M7I3
0,25,362,Fosfatase Ácida Total,1,17.6,4.2,5.3,8.9,9.7,3.8,2.4,5.5,3.3,9.4,2.8,3.7
1,94,362,Fosfatase Ácida Total,1,16.9,3.5,4.9,8.2,9.0,2.8,3.0,5.7,3.9,9.0,3.1,3.6
2,124,362,Fosfatase Ácida Total,1,16.8,3.4,4.2,6.6,7.6,2.7,2.3,4.6,2.7,8.6,2.9,3.4
3,124,362,Fosfatase Ácida Total,2,17.9,4.0,5.2,7.7,9.4,3.2,2.7,5.8,3.7,8.6,3.1,3.5
4,3466,362,Fosfatase Ácida Total,1,NaN,NaN,NaN,8.8,8.9,2.7,2.4,5.6,3.2,8.3,2.5,3.2
5,5596,362,Fosfatase Ácida Total,1,16.6,3.6,4.3,7.5,8.8,2.8,2.1,4.7,3.3,8.5,2.8,3.2
6,6086,362,Fosfatase Ácida Total,1,18.3,3.7,5.0,6.7,7.8,2.1,2.5,5.3,3.0,8.4,3.0,3.2
7,7463,362,Fosfatase Ácida Total,1,NaN,NaN,NaN,7.6,8.5,3.1,2.3,5.3,3.3,8.6,2.9,3.4
8,9379,362,Fosfatase Ácida Total,1,15.9,3.4,4.3,7.5,8.4,3.0,2.5,5.1,3.3,8.0,3.1,3.2
9,9466,362,Fosfatase Ácida Total,1,17.9,4.0,5.2,7.7,9.4,3.2,2.7,5.8,3.7,8.6,3.1,3.5


In [3]:
teste = df_tratado.set_index(['part','id_modulo','analito','sistema'])

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler    
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Preenchendo vazios pela média da linha
imputer = SimpleImputer(strategy="mean")
dados: np.ndarray = imputer.fit_transform(teste.T).T

# Aplicando a normalização
norm = StandardScaler()

dados = norm.fit_transform(dados)
pd.DataFrame(dados)

neighbors = NearestNeighbors(n_neighbors=2)
neighbors_fit = neighbors.fit(dados)
distances, _ = neighbors_fit.kneighbors(dados)
distances = np.mean(distances[:,1:], axis=1)
distances = np.sort(distances)
distances = distances[np.where(distances <= 2)]
slopes = np.diff(distances)
value = slopes[np.where(slopes>=1/100)][[0]]
value
# slopes[np.where(slopes>=1/100)]

index = np.argmax(slopes==value)

eps = distances[index+1]
eps

IndexError: index 0 is out of bounds for axis 0 with size 0